# 6D-MAN Basic Usage Example

This notebook demonstrates the basic usage of the 6D-MAN package for optical metro-urban network planning.

In [13]:
import sys
import os
# Navigate relative to the current working directory
sys.path.append(os.path.abspath('../src'))
import numpy as np
import pandas as pd
from sixdman.core.network import Network
from sixdman.core.band import Band, OpticalParameters
from sixdman.core.planning import PlanningTool
from sixdman.core.visualize import analyse_result
import networkx as nx
import json
from sixdman.utils.paths import get_project_root

import warnings
warnings.filterwarnings("ignore")

In [14]:
# Use it to get paths
project_root = get_project_root()
results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

## 1. Create Network Instance

In [15]:
# Initialize network
network = Network(topology_name = 'MAN157')

# Load topology from .mat file
network.load_topology(filepath = '../data/MAN157Nodes.mat', matrixName ='MAN157Nodes')

# Set hierarchical levels
hl_dict = network.set_hierarchical_levels(
    HL1_standalone = [1, 5],
    HL2_standalone = [0, 2, 3, 4],
    HL3_standalone = list(range(6, 39)),
    HL4_standalone = list(range(39, 157))
)

In [16]:
HL4_Standalone = hl_dict['HL4']['standalone']
HL4_colocated = hl_dict['HL4']['colocated']
HL4_all = np.concatenate((HL4_Standalone, HL4_colocated))

In [17]:
_, subnetMatrix_HL4 = network.calculate_subgraph(hierarchy_level = 4, minimum_hierarchy_level = 4)
HL4_connected_nodes = network.find_neighbors(HL4_Standalone)

Standalone HL4s

In [18]:
file_name = f"./../results/{network.topology_name}_HL4_K_path_attributes.csv"

if os.path.exists(file_name):
    K_path_attributes_df = pd.read_csv(file_name)
    K_path_attributes_df['links'] = K_path_attributes_df['links'].map(json.loads)
    K_path_attributes_df['nodes'] = K_path_attributes_df['nodes'].map(json.loads)
else:
    
    k_paths = 20

    # define a list to store path attributes
    K_path_attributes = []

    # iterate through each standalone HL4 node
    for src in HL4_Standalone:
        for dest in HL4_connected_nodes:
            K_path_attributes = network.calculate_paths(subnetMatrix_HL4, K_path_attributes, source = src, target = dest, k = k_paths)

    # Convert K_path_attributes list to dataframe
    K_path_attributes_df = pd.DataFrame(K_path_attributes)

    # save dataframe to csv file
    K_path_attributes_df.to_csv(file_name, index = False)

Colocated HL4s

In [19]:
file_name = f"./../results/{network.topology_name}_HL4_K_path_attributes_colocated.csv"

if os.path.exists(file_name):
    K_path_attributes_colocated_df = pd.read_csv(file_name)
    K_path_attributes_colocated_df['links'] = K_path_attributes_colocated_df['links'].map(json.loads)
    K_path_attributes_colocated_df['nodes'] = K_path_attributes_colocated_df['nodes'].map(json.loads)
else:
    
    k_paths = 20

    # define a list to store path attributes
    K_path_attributes_colocated = []

    # iterate through each standalone HL4 node
    for src in HL4_colocated:
        for dest in HL4_connected_nodes:
            if src != dest:
                K_path_attributes_colocated = network.calculate_paths(subnetMatrix_HL4, K_path_attributes_colocated, source = src, target = dest, k = k_paths)

    # Convert K_path_attributes list to dataframe
    K_path_attributes_colocated_df = pd.DataFrame(K_path_attributes_colocated)

    # save dataframe to csv file
    K_path_attributes_colocated_df.to_csv(file_name, index = False)

In [20]:
# sort dataframes based on num_hops and distance (in order)
K_path_attributes_df_sorted = K_path_attributes_df.groupby(['src_node'], group_keys = False).apply(lambda x: x.sort_values(['num_hops', 'distance']))
K_path_attributes_colocated_df_sorted = K_path_attributes_colocated_df.groupby(['src_node', 'dest_node'], group_keys = False).apply(lambda x: x.sort_values(['num_hops', 'distance']))

In [21]:
pairs_disjoint = network.land_pair_finder(HL4_Standalone, K_path_attributes_df_sorted, num_pairs = 1)

## 2. Define Transmission Bands

In [22]:
# Define C-band parameters
c_band_params = OpticalParameters()

# Create C-band instance
c_band = Band(
    name='C',
    start_freq = 190.65, # THz
    end_freq = 196.675, # THz
    opt_params = c_band_params,
    network_instance = network,
    channel_spacing = 0.05 # THz
    )

# Define L-band parameters
l_band_params = OpticalParameters()

# Create L-band instance
l_band = Band(
    name='L',
    start_freq = 184.525, # THz
    end_freq = 190.565, # THz
    opt_params = l_band_params,
    network_instance = network,
    channel_spacing = 0.05 # THz
)

In [23]:
# define C-band and L-band frequency slots
spectrum_C = c_band.calc_spectrum()
spectrum_L = l_band.calc_spectrum()

# concatenate C-band and KL-band to a sigle frequency spectrum
spectrum = np.concatenate((spectrum_C, spectrum_L))

# define total number of frequency slots
num_fslots = len(spectrum)

f_c_axis = spectrum * 1e12  # Convert to Hz
Pch_dBm = np.arange(-6, -0.9, 0.1)  # Channel power in dBm
num_Ch_mat = np.arange(1, len(spectrum) - 1)  # Channel indices

In [24]:
GSNR_opt_link, _, _, _ = c_band.process_link_gsnr(f_c_axis = f_c_axis, 
                                                  Pch_dBm = Pch_dBm, 
                                                  num_Ch_mat = num_Ch_mat,
                                                  spectrum_C = spectrum_C,
                                                  Nspan_array = np.ones(network.all_links.shape[0], dtype=int),
                                                  hierarchy_level = 4, 
                                                  minimum_hierarchy_level = 4,
                                                  result_directory = results_dir)

Loading precomputed link GSNR analysis


## 3. Create Planning Tool and Optimize Network

In [25]:
# Initialize planning tool
planner = PlanningTool(
    network_instance = network,
    bands = [c_band, l_band], 
    period_time = 10)


#### *Simulating aggregated traffic at HLs*

In [26]:
planner.initialize_planner(num_fslots = num_fslots,
                           hierarchy_level = 4, 
                           minimum_hierarchy_level = 4)

In [27]:
# generate port capacity for HL4 nodes uisng Monte Carlo simulation
planner.simulate_traffic_initial(num_nodes = len(HL4_all),
                                monteCarlo_steps = 100,
                                min_rate = 20, # Gbps
                                max_rate = 200, # Gbps
                                seed = 20, result_directory = results_dir)

# Traffic growth simulation over 10 years
planner.simulate_traffic_annual(lowest_hierarchy_dict = hl_dict['HL4'],
                                CAGR = 0.4, 
                                result_directory = results_dir)

Loading precomputed HL_capacity_final ...
Loading precomputed Traffic Matrix ...


In [28]:
planner.run_planner(HL_dict = hl_dict['HL4'],
                    pairs_disjoint = pairs_disjoint,
                    kpair_standalone = 1,
                    kpair_colocated = 1,
                    candidate_paths_standalone_df = K_path_attributes_df,
                    candidate_paths_colocated_df = K_path_attributes_colocated_df,
                    GSNR_opt_link = GSNR_opt_link,
                    prev_hierarchy_level = 3,
                    hierarchy_level = 4,
                    minimum_level = 4, 
                    node_cap_update_idx = 2,
                    result_directory = results_dir)

FS_path [0]
FS_path [0]
FS_path [1]
FS_path [1]
FS_path [2]
FS_path [2]
FS_path [3]
FS_path [3]
FS_path [0]
FS_path [4]
FS_path [4]
FS_path [5]
FS_path [5]
FS_path [6]
FS_path [0]
FS_path [0]
FS_path [1]
FS_path [1]
FS_path [2]
FS_path [2]
FS_path [3]
FS_path [3]
FS_path [4]
FS_path [4]
FS_path [5]
FS_path [5]
FS_path [0]
FS_path [6]
FS_path [7]
FS_path [1]
FS_path [2]
FS_path [8]
FS_path [0]
FS_path [9]
FS_path [1]
FS_path [10]
FS_path [0]
FS_path [0]
FS_path [1]
FS_path [1]
FS_path [2]
FS_path [2]
FS_path [3]
FS_path [3]
FS_path [0]
FS_path [0]
FS_path [1]
FS_path [1]
FS_path [2]
FS_path [2]
FS_path [3]
FS_path [3]
FS_path [4]
FS_path [4]
FS_path [5]
FS_path [0]
FS_path [5]
FS_path [6]
FS_path [1]
FS_path [7]
FS_path [0]
FS_path [8]
FS_path [1]
FS_path [9]
FS_path [2]
FS_path [10]
FS_path [0]
FS_path [4]
FS_path [4]
FS_path [5]
FS_path [6]
FS_path [6]
FS_path [0]
FS_path [0]
FS_path [1]
FS_path [1]
FS_path [0]
FS_path [2]
FS_path [0]
FS_path [0]
FS_path [1]
FS_path [1]
FS_path [2]
FS